# 2D -> 3D Data Processing
Execution of this notebook is meant to follow `1_data_processing.ipynb`. 

The 

## Sample the data

In [2]:
import os
import numpy as np
from tqdm import tqdm
import json
import cv2

# Constants
from __init__ import data_path
from data_utils import read_segment_frames, read_segment_labels, visualize_frame_annotations, visualize_segment_labels
from PoseLifter import PoseLifter
# Choose dataset
dataset_path = "/home/florsanders/adl_ai_tennis_coach/data/tenniset"
write_path = "/home/georgetamer/3d_poses"
segments_path = os.path.join(dataset_path, "segments")
labels_path = os.path.join(dataset_path,"labels")

In [ ]:
def crop_frame(frame,
              bbox,
              crop_padding=50,
            crop_img_width=256):
    # Frame size
    frame_height, frame_width = frame.shape[:2]

    # Parse bounding box coords
    if np.any(bbox == None):
        return best_keypoints, best_bbox
    x1, y1, x2, y2 = bbox
    xc, yc =  (x1 + x2) / 2, (y1 + y2) / 2
    w, h = abs(x2 - x1), abs(y2 - y1)
    d = max(w, h) + crop_padding * 2

    # Define cropping indices
    x_crop1, x_crop2 = int(xc - d/2), int(xc + d/2)
    y_crop1, y_crop2 = int(yc - d/2), int(yc + d/2)

    # Make sure we don't crop past the edges of the frame
    x_crop_offset = min(frame_width - x_crop2, max(-x_crop1, 0))
    y_crop_offset = min(frame_height - y_crop2, max(-y_crop1, 0))
    x_crop1 += x_crop_offset
    x_crop2 += x_crop_offset
    y_crop1 += y_crop_offset
    y_crop2 += y_crop_offset
    
    # Crop image
    img = frame[y_crop1:y_crop2,  x_crop1:x_crop2].copy()

    # Resize img
    scale = d / crop_img_width
    img = cv2.resize(img, (crop_img_width, crop_img_width))
    return img

def detect_pose(
    frame,
    bbox,
    crop_padding=50,
    crop_img_width=256
):

    # Crop image
    crop_padding=10
    cropped_frame = crop_frame(frame, bbox, crop_padding,crop_img_width)

    # Detect pose
    should_show = False
    result_generator = inferencer(cropped_frame, show=should_show, vis_out_dir="/home/georgetamer/3d_poses", return_vis=True)
    results = [result for result in result_generator]

    # # Parse keypoints
    # min_center_distance = np.inf
    # result = results[0]
    # prediction = result["predictions"][0]
    # item = prediction[0]
    # keypoints = item["keypoints"]

    # Keep best results
    min_center_distance = np.inf
    for result in results:
        print("result: ", result)
        for prediction in result["predictions"]:
            for item in prediction:
                # Parse item
                # pose_bbox = item["bbox"]
                keypoints = item["keypoints"]

                # Parse bbox
                # xb1, yb1, xb2, yb2 = pose_bbox[0]
                # center_x = (xb1 + xb2) / 2
                # center_y = (yb1 + yb2) / 2
                # center_distance = ((crop_img_width / 2 - center_x)**2  + (crop_img_width / 2 - center_y)**2)**(1/2)

                # # Keep track of best prediction
                # if center_distance < min_center_distance:
                #     min_center_distance = center_distance
                #     best_keypoints = keypoints
                #     best_bbox = pose_bbox[0]

    return np.array(keypoints)

In [ ]:
pose_lifter = PoseLifter(
    crop_fn=crop_frame,
    dedup_heuristic_fn="TODO"
)

pose_lifter.set_dataset_path(dataset_path)
pose_lifter.set_write_path(write_path)

In [ ]:
pose_lifter.extract_3d_poses()

In [ ]:
seg_path = os.path.join(segments_path, "V009_0061.mp4")
print("Segment path", seg_path)

success = process_segment(
    segment_path=seg_path, 
    labels_path=labels_path)

In [ ]:
# visualize
from mmpose.apis import visualize

# extract first frame from segment
img_path = os.path.join(write_path, '000000.jpg')
keypoints = np.load(sample_file, allow_pickle=True)
keypoint_scores = None

metainfo = 'config/_base_/datasets/coco.py'

visualize(
    img_path,
    keypoints,
    keypoint_scores,
    metainfo=metainfo,
    show=True)

In [ ]:
# take heuristic such as max area, or largest x-span, y-span
